In [3]:
import os
import pandas as pd
import json

# Define the path to the folder containing JSON files
folder_path = '/Users/kritigupta/mlp project/Article-Bias-Prediction/data/jsons'

# List all JSON files in the folder
json_files = [pos_json for pos_json in os.listdir(folder_path) if pos_json.endswith('.json')]

# Initialize an empty list to store data
data_list = []

# Loop through each JSON file and read its content
for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    with open(file_path, 'r') as file:
        data = json.load(file)
        data_list.append(data)

# Convert the list of JSON data to a DataFrame
df = pd.DataFrame(data_list)

# # Display the DataFrame
# print(df)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset as HFDataset

/Users/kritigupta/miniconda3/envs/mlp_proj/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_dataset = df.iloc[:round(0.8*len(df))]
valid_dataset = df.iloc[round(0.8*len(df)):round(0.9*len(df))]
test_dataset = df.iloc[round(0.9*len(df)):]

In [6]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [7]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)


In [6]:
train_encodings = tokenize_function(train_dataset['content'].to_list())
valid_encodings = tokenize_function(valid_dataset['content'].to_list())
test_encodings = tokenize_function(test_dataset['content'].to_list())

In [13]:
train_dataset = HFDataset.from_dict({"text": train_dataset['content'].to_list(), "label": train_dataset['bias'].to_list()})
val_dataset = HFDataset.from_dict({"text": valid_dataset['content'].to_list(), "label": valid_dataset['bias'].to_list()})
test_dataset = HFDataset.from_dict({"text": test_dataset['content'].to_list(), "label": test_dataset['bias'].to_list()})

In [14]:
train_dataset = train_dataset.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True)
val_dataset = val_dataset.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True)
test_dataset = test_dataset.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=512), batched=True)

Map: 100%|██████████| 3755/3755 [00:09<00:00, 378.32 examples/s]


In [15]:
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

In [16]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    max_grad_norm=1.0,
    fp16=False
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

/var/folders/f6/733_g2413bg21q9jq89mmyc00000gn/T/ipykernel_5979/3286434519.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

  0%|          | 10/11268 [00:18<5:21:46,  1.71s/it]

{'loss': 1.101, 'grad_norm': 2.180868148803711, 'learning_rate': 4.9955626553070645e-06, 'epoch': 0.0}


  0%|          | 20/11268 [00:35<5:15:27,  1.68s/it]

{'loss': 1.1047, 'grad_norm': 20.548362731933594, 'learning_rate': 4.991125310614129e-06, 'epoch': 0.01}


  0%|          | 30/11268 [00:52<5:22:15,  1.72s/it]

...

                                                       
100%|██████████| 11268/11268 [9:09:47<00:00,  2.17s/it]

{'eval_loss': 0.4616864025592804, 'eval_runtime': 379.2874, 'eval_samples_per_second': 9.903, 'eval_steps_per_second': 1.239, 'epoch': 3.0}


100%|██████████| 11268/11268 [9:09:51<00:00,  2.93s/it]

{'train_runtime': 32991.3852, 'train_samples_per_second': 2.732, 'train_steps_per_second': 0.342, 'train_loss': 0.43026263869566234, 'epoch': 3.0}


TrainOutput(global_step=11268, training_loss=0.43026263869566234, metrics={'train_runtime': 32991.3852, 'train_samples_per_second': 2.732, 'train_steps_per_second': 0.342, 'total_flos': 2.371414922640691e+16, 'train_loss': 0.43026263869566234, 'epoch': 3.0})

epoch 3 model

In [9]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_path = "./results/checkpoint-11268" 
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [11]:
test_dataset = df.iloc[round(0.9*len(df)):]
batch_size = 50
all_predictions = []
for i in range(0, len(test_dataset), batch_size):
    print(i)
    temp = test_dataset.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())


# test_encodings = tokenizer(test_dataset['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750


In [15]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
# convert all preds to numpy
np_preds = np.array(all_predictions)
labels = test_dataset["bias"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.8631
              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1299
           1       0.88      0.85      0.86      1109
           2       0.86      0.86      0.86      1347

    accuracy                           0.86      3755
   macro avg       0.86      0.86      0.86      3755
weighted avg       0.86      0.86      0.86      3755



In [16]:
valid_dataset = df.iloc[round(0.8*len(df)):round(0.9*len(df))]
batch_size = 50
all_predictions = []
for i in range(0, len(test_dataset), batch_size):
    print(i)
    temp = valid_dataset.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())


# test_encodings = tokenizer(test_dataset['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750


In [18]:
np_preds = np.array(all_predictions)
labels = valid_dataset["bias"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.8698
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1310
           1       0.87      0.86      0.86      1061
           2       0.89      0.86      0.87      1385

    accuracy                           0.87      3756
   macro avg       0.87      0.87      0.87      3756
weighted avg       0.87      0.87      0.87      3756



epoch 2 model

In [19]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_path = "./results/checkpoint-7512" 
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [20]:
test_dataset = df.iloc[round(0.9*len(df)):]
batch_size = 50
all_predictions = []
for i in range(0, len(test_dataset), batch_size):
    print(i)
    temp = test_dataset.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750


In [22]:
np_preds = np.array(all_predictions)
labels = test_dataset["bias"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.8453
              precision    recall  f1-score   support

           0       0.80      0.90      0.85      1299
           1       0.89      0.83      0.86      1109
           2       0.87      0.80      0.83      1347

    accuracy                           0.85      3755
   macro avg       0.85      0.85      0.85      3755
weighted avg       0.85      0.85      0.85      3755



epoch 1 model

In [5]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_path = "./results/checkpoint-3756" 
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [24]:
test_dataset = df.iloc[round(0.9*len(df)):]
batch_size = 50
all_predictions = []
for i in range(0, len(test_dataset), batch_size):
    print(i)
    temp = test_dataset.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['content'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750


In [25]:
np_preds = np.array(all_predictions)
labels = test_dataset["bias"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.8413
              precision    recall  f1-score   support

           0       0.87      0.84      0.85      1299
           1       0.85      0.81      0.83      1109
           2       0.81      0.87      0.84      1347

    accuracy                           0.84      3755
   macro avg       0.84      0.84      0.84      3755
weighted avg       0.84      0.84      0.84      3755



testing on twin views

In [8]:
babe_test_df = pd.read_excel("/Users/kritigupta/mlp project/data/final_labels_MBIC.xlsx")
babe_test_df.head()

/Users/kritigupta/miniconda3/envs/mlp_proj/lib/python3.8/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,text,news_link,outlet,topic,type,group_id,num_sent,label_bias,label_opinion,article,biased_words
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1,1,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
2,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1,1,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."
3,The Republican president assumed he was helpin...,http://www.msnbc.com/rachel-maddow-show/auto-i...,msnbc,environment,left,1,1,Biased,Expresses writer’s opinion,"In Barack Obama’s first term, the administrati...","['rejects', 'happy', 'assumed']"
4,The explosion of the Hispanic population has l...,https://www.breitbart.com/politics/2015/02/26/...,breitbart,student-debt,right,1,1,Biased,No agreement,"Republicans should stop fighting amnesty, Pres...",['explosion']


In [10]:
# converting type to integer label
babe_test_df['label'] = babe_test_df['type'].map({'left': 0, 'center': 1, 'right': 2})


In [13]:
# drop duplicates from text column
babe_test_df = babe_test_df.drop_duplicates(subset=['text'])

#drop empty topic rows
babe_test_df = babe_test_df.dropna(subset=['text','topic'])

babe_test_df.shape

(1700, 12)

In [14]:
batch_size = 50
all_predictions = []
for i in range(0, len(babe_test_df), batch_size):
    print(i)
    temp = babe_test_df.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['text'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650


In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
np_preds = np.array(all_predictions)
labels = babe_test_df["label"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.5006
              precision    recall  f1-score   support

           0       0.45      0.86      0.59       694
           1       0.80      0.01      0.03       315
           2       0.66      0.36      0.46       691

    accuracy                           0.50      1700
   macro avg       0.64      0.41      0.36      1700
weighted avg       0.60      0.50      0.44      1700



In [18]:
df_twinviews_test = df = pd.read_csv("hf://datasets/wwbrannon/twinviews-13k/twinviews-13k.csv")
df_twinviews_test.head()

,l,r,topic
0,LGBTQ+ individuals should have the same rights...,Marriage should only be between a man and a wo...,LGBTQ+ Rights
1,Police reform is necessary to address systemic...,Law enforcement should be supported and given ...,Police Reform
2,A woman should have the right to choose whethe...,Abortion should be illegal and considered as t...,Abortion Rights
3,Increase regulations on businesses to protect ...,Reduce regulations on businesses to promote in...,Regulation
4,Investing in clean energy technologies will cr...,Government subsidies for clean energy distort ...,Clean Energy


In [19]:
# drop topic duplicates (taking only one sentence per topic)
df_twinviews_test['topic']  = df_twinviews_test.topic.str.lower()
df_twinviews_test = df_twinviews_test.drop_duplicates(subset=['topic'])


In [20]:
df_twinviews_test.shape

(875, 3)

In [26]:
#split dataset by l and r
d1 = df_twinviews_test[['l','topic']]
d1['label'] = 0
d1.rename(columns={'l':'text'},inplace=True)
d2 = df_twinviews_test[['r','topic']]
d2['label'] = 2
d2.rename(columns={'r':'text'},inplace=True)
df_twinviews_test = pd.concat([d1,d2])

KeyError: "['l'] not in index"

In [31]:
df_twinviews_test.drop_duplicates(subset=['text'],inplace=True)
df_twinviews_test.dropna(subset=['text'],inplace=True)

In [32]:
df_twinviews_test.shape

(1745, 3)

In [33]:
batch_size = 50
all_predictions = []
for i in range(0, len(df_twinviews_test), batch_size):
    print(i)
    temp = df_twinviews_test.iloc[i:i+batch_size]
    test_encodings = tokenizer(temp['text'].to_list(), padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        predictions = model(**test_encodings)
    all_predictions.extend(predictions.logits.argmax(dim=-1).cpu().numpy())

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700


In [34]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
np_preds = np.array(all_predictions)
labels = df_twinviews_test["label"].to_numpy()
print(f"Accuracy: {accuracy_score(labels, np_preds):.4f}")
print(classification_report(labels, np_preds))

Accuracy: 0.6126
              precision    recall  f1-score   support

           0       0.58      0.81      0.68       872
           1       0.00      0.00      0.00         0
           2       0.69      0.41      0.52       873

    accuracy                           0.61      1745
   macro avg       0.43      0.41      0.40      1745
weighted avg       0.64      0.61      0.60      1745



/Users/kritigupta/miniconda3/envs/mlp_proj/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kritigupta/miniconda3/envs/mlp_proj/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kritigupta/miniconda3/envs/mlp_proj/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu